In [5]:
import pickle
import altair as alt

import huspacy
import pandas as pd

from bertopic import BERTopic
from hdbscan import HDBSCAN
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance
from sentence_transformers import SentenceTransformer

In [6]:
meta = pickle.load(open("../resources/meta.pkl", "rb"))
lemmatized = pickle.load(open("../resources/lemmatized.pkl", "rb"))
#pos = pickle.load(open("../resources/pos.pkl", "rb"))
#tokens =  pickle.load(open("../resources/tokenized.pkl", "rb"))
#doc_stop = pickle.load(open("../resources/no_stopword.pkl", "rb")) ### kevesebb stopszóval
doc_stop_2 = pickle.load(open("../resources/stopword_filtered.pkl", "rb")) ### több stop szóval, a no_stopword.pkl kiegészítve
all_docs = pickle.load(open("../resources/docs.pkl", "rb"))

In [7]:
prob_matrix = pd.read_csv("../results/prob_matrix.csv", sep=",")
prob_matrix["idx"] = list(range(prob_matrix.shape[0]))
prob_matrix.drop(columns=["25"], inplace=True)

In [8]:
import numpy as np

discourse_matrix = []

filenames = list(meta.values())
filenames = set([e.split(".txt")[0] for e in filenames])
for filename in filenames:
    file_meta = [(k,v) for k,v in meta.items() if filename in v]
    vals = [e[0] for e in file_meta]
    disc_prob_matrix = prob_matrix[prob_matrix["idx"].isin(vals)]
    disc_prob_matrix.drop(columns=["idx"], inplace=True)
    #save each discourse matrix to a separate csv file with the filename
    disc_prob_matrix.to_csv(f"../results/discourse_matrices/{filename}.csv", sep=",", encoding="UTF-8")
    m = np.mean(disc_prob_matrix.values, axis=0)
    discourse_matrix.append(m)

In [9]:
discourse_matrix = np.asarray(discourse_matrix)
discourse_df = pd.DataFrame(data=discourse_matrix,
                            index=list(range(discourse_matrix.shape[0])),
                            columns=list(range(25)))

In [10]:
discourse_df

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.232671,0.133585,0.056717,0.028874,0.039458,0.020153,0.005691,0.005015,0.006812,0.022852,...,0.038828,0.004079,0.017834,0.010937,0.014864,0.009222,0.012920,0.004180,0.013024,0.008725
1,0.201424,0.156990,0.040744,0.063463,0.029189,0.022579,0.004853,0.015004,0.008987,0.032636,...,0.012081,0.022173,0.011614,0.020088,0.008599,0.008081,0.010710,0.017895,0.010773,0.007096
2,0.239162,0.116873,0.040496,0.014145,0.030621,0.012100,0.024798,0.007848,0.010321,0.026455,...,0.030502,0.003676,0.015684,0.014771,0.014195,0.015900,0.012083,0.003755,0.016571,0.008185
3,0.174445,0.129968,0.163121,0.015643,0.038824,0.012097,0.011263,0.009070,0.004110,0.011355,...,0.012264,0.004207,0.011756,0.039937,0.014884,0.010473,0.010049,0.004298,0.009944,0.021691
4,0.259735,0.125167,0.085053,0.026228,0.027145,0.014041,0.011760,0.012646,0.031816,0.017215,...,0.015266,0.003838,0.013649,0.010534,0.010406,0.012163,0.012299,0.008240,0.013524,0.015160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,0.224840,0.112454,0.061849,0.036331,0.024719,0.019304,0.013983,0.006248,0.023726,0.010764,...,0.012085,0.012423,0.011271,0.035003,0.008492,0.006735,0.009615,0.009008,0.009712,0.006420
120,0.153587,0.200281,0.046232,0.061896,0.047120,0.019467,0.016958,0.003985,0.006719,0.013787,...,0.014719,0.004334,0.015261,0.009709,0.039809,0.009398,0.012698,0.004462,0.012785,0.008060
121,0.272556,0.139856,0.051230,0.057923,0.049916,0.013106,0.012682,0.004400,0.022753,0.018807,...,0.027436,0.004587,0.017513,0.011730,0.018740,0.010007,0.014372,0.004709,0.014668,0.009200
122,0.184070,0.156355,0.090507,0.043838,0.032318,0.017027,0.019021,0.003777,0.012634,0.025135,...,0.022308,0.003744,0.017379,0.009163,0.022618,0.008563,0.014844,0.003834,0.010725,0.007560


In [11]:
topics=pd.read_csv("../results/bert_probabilities_merged_85_new_names.csv",sep=",")

In [12]:
topic_names = topics["CustomName"]
topic_names

0                                         diskurzuselem
1                                   helyeslés, hümmögés
2                                               nevetés
3                     tanul, nyelv, angol, egyetem, nem
4                               óra, idő, nem, tud, nap
5           barát, ismerkedés, ember, egymás, kapcsolat
6                   olvas, könyv, elolvas, szeret, film
7         magyarorsz, magyar, olaszorszá, képzel, olasz
8                      sportol, sport, foci, edzés, nem
9                      éves, család, gyerek, nem, szülő
10                  német, spanyol, angol, tanul, olasz
11                         év, nyár, félév, tavaly, két
12             külföld, külföldi, tanulás, megy, utazik
13               macska, eszik, kutya, gyümölcs, szeret
14                     film, sorozat, néz, évad, horror
15                         tíz, három, négy, kilenc, év
16    facebook, instagram, használ, facebookon, mess...
17                    szoba, otthon, itthon, ház

In [13]:
discourse_df["diskurzus"] = list(range(discourse_matrix.shape[0]))
df_melted = pd.melt(
    discourse_df,
    id_vars=["diskurzus"],
    value_vars=list(range(25))
)

In [14]:
df_melted["topik"] = df_melted["variable"]
df_melted.drop(columns=["variable"], inplace=True)

In [15]:
df_melted.head()

,diskurzus,value,topik
0,0,0.232671,0
1,1,0.201424,0
2,2,0.239162,0
3,3,0.174445,0
4,4,0.259735,0


In [16]:
df_melted["topik_nev"] = df_melted["topik"].apply(lambda x: topic_names[x])
df_melted.head()

,diskurzus,value,topik,topik_nev
0,0,0.232671,0,diskurzuselem
1,1,0.201424,0,diskurzuselem
2,2,0.239162,0,diskurzuselem
3,3,0.174445,0,diskurzuselem
4,4,0.259735,0,diskurzuselem


In [17]:
df_melted.to_csv("../results/melted_discourse_probs.csv",sep=",", encoding="UTF-8")